# Drawing on Live Camera

We'll be drawing on each video frame.

In [ ]:
import os
import tempfile
import subprocess
import cv2
import numpy as np

# FFMPEG_BIN = '/usr/bin/ffmpeg'
# To get this path execute:
#    $ which ffmpeg
FFMPEG_BIN = '/home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg'

# to find allowed formats 
#    $ ffmpeg -f v4l2 -list_formats all -i /dev/video3
#    ...
#    [video4linux2,v4l2 @ 0x5608ac90af40] Raw: yuyv422: YUYV 4:2:2: 640x480 1280x720 960x544 800x448 640x360 424x240 352x288 320x240 800x600 176x144 160x120 1280x800
#    ...

width = 640
height = 480

# /home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg -i /dev/video2 -video_size 640x480 -r 1 -pix_fmt bgr24 -vcodec rawvideo -an -sn -f v4l2
def run_ffmpeg():
    ffmpg_cmd = [
        FFMPEG_BIN,
        '-i', '/dev/video2',
        '-video_size', f'{width}x{height}',
        '-pix_fmt', 'bgr24',        # opencv requires bgr24 pixel format. 'yuyv422'
        '-vcodec', 'rawvideo',
        '-an','-sn',                # disable audio processing
        '-f', 'image2pipe',
        '-',                        # output to go to stdout
    ]
    return subprocess.Popen(ffmpg_cmd, stdout = subprocess.PIPE, bufsize=10**8)

# top left corner
x1 = width // 2 # use // to get int
y1 = height // 2

# width and height of the rectangle
w = width // 4
h = height // 4

# bottom right corner
x2 = x1 + w
y2 = y1 + h

def run_cv_window(process):
    while True:
        raw_image = process.stdout.read(640*480*3)
        if raw_image == b'':
            raise RuntimeError("Empty pipe")
        
        frame =  np.frombuffer(raw_image, dtype='uint8')
        frame = frame.reshape((height,width,3))          # height, width, colours
        if frame is not None:
            # drawing a rectangle on the frame image
            cv2.rectangle(frame, (x1, y1), (x2, y2), color = (0, 0, 255), thickness=4)
            cv2.imshow('Video', frame) # uncomment to show the original image        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cv2.destroyAllWindows()
    process.terminate()

    try:
        outs, errs = process.communicate(timeout=5)
    except TimeoutExpired:
        process.kill()
        outs, errs = process.communicate()
    
    print(process.poll())

def run():
    ffmpeg_process = run_ffmpeg()
    run_cv_window(ffmpeg_process)

run()

## How to interactively draw on video

In [2]:
import os
import tempfile
import subprocess
import cv2
import numpy as np

# FFMPEG_BIN = '/usr/bin/ffmpeg'
# To get this path execute:
#    $ which ffmpeg
FFMPEG_BIN = '/home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg'

# to find allowed formats 
#    $ ffmpeg -f v4l2 -list_formats all -i /dev/video3
#    ...
#    [video4linux2,v4l2 @ 0x5608ac90af40] Raw: yuyv422: YUYV 4:2:2: 640x480 1280x720 960x544 800x448 640x360 424x240 352x288 320x240 800x600 176x144 160x120 1280x800
#    ...

width = 640
height = 480

pt1 = (0, 0) # top left
pt2 = (0, 0) # bottom right
topLeft_clicked = False
botRight_clicked = False

# /home/bojan/anaconda3/envs/python-cvcourse/bin/ffmpeg -i /dev/video2 -video_size 640x480 -r 1 -pix_fmt bgr24 -vcodec rawvideo -an -sn -f v4l2
def run_ffmpeg():
    ffmpg_cmd = [
        FFMPEG_BIN,
        '-i', '/dev/video2',
        '-video_size', f'{width}x{height}',
        '-pix_fmt', 'bgr24',        # opencv requires bgr24 pixel format. 'yuyv422'
        '-vcodec', 'rawvideo',
        '-an','-sn',                # disable audio processing
        '-f', 'image2pipe',
        '-',                        # output to go to stdout
    ]
    return subprocess.Popen(ffmpg_cmd, stdout = subprocess.PIPE, bufsize=10**8)
   
# callback function
def draw_rectangle(e, x, y, flags, param):
    global pt1, pt2, topLeft_clicked, botRight_clicked

    if e == cv2.EVENT_LBUTTONDOWN:
        # reset the global vars if rect has already been drawn
        if topLeft_clicked and botRight_clicked:
            pt1 = (0, 0)
            pt2 = (0, 0) 
            topLeft_clicked = False
            botRight_clicked = False
        if topLeft_clicked == False:
            pt1 = (x, y)
            topLeft_clicked = True
        elif botRight_clicked == False:
            pt2 = (x, y)
            botRight_clicked = True
       
# connect to callback
winName = 'Drawing on video'
cv2.namedWindow(winName)
cv2.setMouseCallback(winName, draw_rectangle)

def run_cv_window(process):
    while True:
        raw_image = process.stdout.read(640*480*3)
        if raw_image == b'':
            raise RuntimeError("Empty pipe")
        
        frame =  np.frombuffer(raw_image, dtype='uint8')
        frame = frame.reshape((height,width,3))          # height, width, colours
        if frame is not None:
            # drawing on the frame image
            if topLeft_clicked:
                if botRight_clicked:
                    cv2.rectangle(frame, pt1, pt2, (0,0,255), 3)
                else:
                    cv2.circle(frame, center = pt1, radius=5, color=(0,0,255), thickness=-1)                           
       
            cv2.imshow(winName, frame) # uncomment to show the original image        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cv2.destroyAllWindows()
    process.terminate()

    try:
        outs, errs = process.communicate(timeout=5)
    except TimeoutExpired:
        process.kill()
        outs, errs = process.communicate()
    
    print(process.poll())

def run():
    ffmpeg_process = run_ffmpeg()
    run_cv_window(ffmpeg_process)

run()

255
